<a href="https://colab.research.google.com/github/JulianSantos-LATAMAI/ECON-5200/blob/main/Lab_9/Lab_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

# Load your dataset here (ensure lalonde.csv is uploaded to Colab or linked)
df = pd.read_csv('lalonde.csv')

In [6]:
# Naive Comparison
naive_diff = df[df['treat'] == 1]['re78'].mean()
print(f"Naive Difference in Means: ${naive_diff:,.2f}")
# Expected Result: -$635.03

Naive Difference in Means: $6,349.14


In [7]:
print (df.head())

   Unnamed: 0  treat  age  educ  black  hispan  married  nodegree  re74  re75  \
0           1      1   37    11      1       0        1         1   0.0   0.0   
1           2      1   22     9      0       1        0         1   0.0   0.0   
2           3      1   30    12      1       0        0         0   0.0   0.0   
3           4      1   27    11      1       0        0         1   0.0   0.0   
4           5      1   33     8      1       0        0         1   0.0   0.0   

         re78  
0   9930.0460  
1   3595.8940  
2  24909.4500  
3   7506.1460  
4    289.7899  


In [9]:
# Define covariates
X = df [['age', 'educ' , 'black', 'hispan', 'married', 'nodegree', 're74', 're75']]
y = df['treat']

# Fit Propensity Model
logit = LogisticRegression(solver='liblinear')
logit.fit(X, y)

# Generate Scores
df['pscore'] = logit.predict_proba(X)[:, 1]
#View the first propensity scores
df['pscore'].head()

,pscore
0,0.443350
1,0.144660
2,0.722355
3,0.664151
4,0.698286


In [16]:
from sklearn.neighbors import NearestNeighbors

# Separate groups
treated = df[df.treat==1]
control = df[df.treat==0]

# Fit NN on Control scores
nbrs = NearestNeighbors(n_neighbors=1).fit(control[['pscore']])

# Find matches for Treated scores
distances, indices = nbrs.kneighbors(treated[['pscore']])
matched_control = control.iloc[indices.flatten()]

# Construct Matched DataFrame
matched_df = pd.concat([treated, matched_control])


In [17]:
from scipy import stats

# T-test on raw data
diff = treated['re78'].mean() - control['re78'].mean()
t_stat, p_val = stats.ttest_ind(treated['re78'], control['re78'])

print(f"Raw Effect (Difference): ${diff:,.2f}")
print(f"P-value: {p_val:.4f}")


# Isolate the matched outcomes
matched_treated = matched_df[matched_df.treat==1]['re78']
matched_control = matched_df[matched_df.treat==0]['re78']

# Estimate the causal effect (T-test on matched data)
matched_diff = matched_treated.mean() - matched_control.mean()
t_stat, p_val = stats.ttest_ind(matched_treated, matched_control)


print(f"Recovered Effect (Matched Difference): ${matched_diff:,.2f}")
print(f"P-value: {p_val:.4f}")


Raw Effect (Difference): $-635.03
P-value: 0.3342
Recovered Effect (Matched Difference): $583.04
P-value: 0.4438
